#### Specify NetCDF files used for training and prediciton inputs
These are low resolution versions of NCAR CAM inputs/outputs, located in the `example_data` directory of this project's git repository.

In [35]:
data_dir = "/home/adamsjam/git/model_learn/example_data"
result_dir = "/home/adamsjam/test"

# files used as feature inputs for model training
netcdf_features_train = [data_dir + "/fv091x180L26_moist_HS.cam.h0.2001-01-11-00000_lowres.nc",
                         data_dir + "/fv091x180L26_moist_HS.cam.h0.2001-01-26-00000_lowres.nc",
                         data_dir + "/fv091x180L26_moist_HS.cam.h0.2001-02-10-00000_lowres.nc"]

# files used as label inputs for model training
netcdf_labels_train = [data_dir + "/fv091x180L26_moist_HS.cam.h1.2001-01-11-00000_lowres.nc",
                       data_dir + "/fv091x180L26_moist_HS.cam.h1.2001-01-26-00000_lowres.nc",
                       data_dir + "/fv091x180L26_moist_HS.cam.h1.2001-02-10-00000_lowres.nc"]

# files used as feature inputs for model prediction
netcdf_features_predict = [data_dir + "/fv091x180L26_moist_HS.cam.h0.2001-02-25-00000_lowres.nc"]

# files used as label outputs for model prediction
netcdf_predict = [result_dir + "/fv091x180L26_moist_HS.cam.h1.2001-02-25-00000_lowres_predicted.nc"]

### Load datasets for training and prediction

We'll define a function to extract an array of variable(s) for a single level from an xarray DataSet, and another to extract both features and labels from NetCDF.

In [2]:
import numpy as np
import xarray as xr

def extract_data_array(dataset,
                       variables,
                       lev):

    # allocate the array
    arr = np.empty(shape=[dataset.time.size, 
                          dataset.lat.size, 
                          dataset.lon.size, 
                          len(variables)],
                   dtype=np.float64)
    
    # for each variable we'll extract the values 
    for var_index, var in enumerate(variables):

        # if we have (time, lev, lat, lon), then use level parameter
        dimensions = dataset.variables[var].dims
        if dimensions == ('time', 'lev', 'lat', 'lon'):
            values = dataset[var].values[:, lev, :, :]
        elif dimensions == ('time', 'lat', 'lon'):
            values = dataset[var].values[:, :, :]
        else:
            raise ValueError("Unsupported variable dimensions: {dims}".format(dims=dimensions))

        # add the values into the array at the variable's position
        arr[:, :, :, var_index] = values
    
    return arr
    
    
def extract_features_labels(netdcf_features, 
                            netcdf_labels,
                            feature_vars,
                            label_vars,
                            level=0):
    """
    Extracts feature and label data from specified NetCDF files for a single level as numpy arrays.
    
    The feature and label NetCDFs are expected to have matching time, level, lat, and lon coordinate variables.
    
    Returns two arrays: the first for features and the second for labels. Arrays will have shape (time, lat, lon, var),
    where var is the number of feature or label variables. For example if the dimensions of feature data variables in 
    the NetCDF is (time: 360, lev: 26, lat: 120, lon: 180) and the features specified are ["T", "U"] then the resulting
    features array will have shape (360, 120, 180, 2), with the first feature variable "T" corresponding to array[:, :, :, 0]
    and the second feature variable "U" corresponding to array[:, :, :, 1].
    
    :param netdcf_features: one or more NetCDF files containing feature variables, can be single file or list
    :param netdcf_features: one or more NetCDF files containing label variables, can be single file or list
    :param feature_vars: list of feature variable names to be extracted from the features NetCDF
    :param label_vars: list of label variable names to be extracted from the labels NetCDF
    :param level: index of the level to be extracted (all times/lats/lons at this level for each feature/label variable)
    :return: two 4-D numpy arrays, the first for features and the second for labels
    """
    
    # open the features (flows) and labels (tendencies) as xarray DataSets
    ds_features = xr.open_mfdataset(paths=netdcf_features)
    ds_labels = xr.open_mfdataset(paths=netcdf_labels)

    # confirm that we have datasets that match on the time, lev, lat, and lon dimension/coordinate
    if np.any(ds_features.variables['time'].values != ds_labels.variables['time'].values):
        raise ValueError('Non-matching time values between feature and label datasets')
    if np.any(ds_features.variables['lev'].values != ds_labels.variables['lev'].values):
        raise ValueError('Non-matching level values between feature and label datasets')
    if np.any(ds_features.variables['lat'].values != ds_labels.variables['lat'].values):
        raise ValueError('Non-matching lat values between feature and label datasets')
    if np.any(ds_features.variables['lon'].values != ds_labels.variables['lon'].values):
        raise ValueError('Non-matching lon values between feature and label datasets')

    # extract feature and label arrays at the specified level
    array_features = extract_data_array(ds_features, feature_vars, level)
    array_labels = extract_data_array(ds_labels, label_vars, level)
    
    return array_features, array_labels

The feature variables being used are 'PS', 'T', 'U', and 'V'. The label variable is 'PTTEND'.

In [3]:
features = ["PS", "T", "U", "V"]
labels = ["PTTEND"]

Read NetCDF files to load feature and label datasets that will be used for training and prediction.

All files should share the same time, level, lat, and lon coordinate, with each file's feature or label variables having shape (times, levels, lats, lons).

In [28]:
train_x, train_y = extract_features_labels(netcdf_features_train[0],
                                           netcdf_labels_train[0],
                                           features,
                                           labels,
                                           level=0)
predict_x = extract_data_array(xr.open_dataset(netcdf_features_predict[0]),
                               features,
                               lev=0)

Get the dimension sizes for later use.

In [29]:
size_times_train = train_x.shape[0]
size_times_predict = predict_x.shape[0]
size_lat = train_x.shape[1]
size_lon = train_x.shape[2]
size_lev = xr.open_dataset(netcdf_features_predict[0]).lev.size

Neural network models work much better if all values are scaled into a range such as between 0 and 1. For this purpose we'll use scikit-learn's MinMaxScaler for now. The scaler being used for labels will be reused later for inverse scaling of the predicted label values.

In [30]:
from sklearn.preprocessing import MinMaxScaler

# initialize a list to store scalers for each feature/label
scalers_x = [MinMaxScaler(feature_range=(0, 1))] * len(features)
scalers_y = [MinMaxScaler(feature_range=(0, 1))] * len(labels)

# function to perform scaling
def scale_4d(features_train,
             features_predict,
             labels_train,
             feature_scalers,
             label_scalers):
    
    # make new arrays to contain the scaled values we'll return
    scaled_features_train = np.empty(shape=features_train.shape)
    scaled_features_predict = np.empty(shape=features_predict.shape)
    scaled_labels_train = np.empty(shape=labels_train.shape)
    
    # data is 4-D with shape (times, lats, lons, vars), scalers can only work on 2-D arrays,
    # so for each feature we scale the corresponding 3-D array of values after flattening it,
    # then reshape back into the original shape
    for feature_ix in range(features_train.shape[-1]):
        scaler = feature_scalers[feature_ix]
        feature_train = features_train[:, :, :, feature_ix].flatten().reshape(-1, 1)
        feature_predict = features_predict[:, :, :, feature_ix].flatten().reshape(-1, 1)
        scaled_train = scaler.fit_transform(feature_train)
        scaled_predict = scaler.fit_transform(feature_predict)
        reshaped_scaled_train = np.reshape(scaled_train, newshape=(size_times_train, size_lat, size_lon))
        reshaped_scaled_predict = np.reshape(scaled_predict, newshape=(size_times_predict, size_lat, size_lon))
        scaled_features_train[:, :, :, feature_ix] = reshaped_scaled_train
        scaled_features_predict[:, :, :, feature_ix] = reshaped_scaled_predict
    for label_ix in range(len(labels)):
        scaler = label_scalers[label_ix]
        label_train = labels_train[:, :, :, label_ix].flatten().reshape(-1, 1)
        scaled_train = scaler.fit_transform(label_train)
        reshaped_scaled_train = np.reshape(scaled_train, newshape=(size_times_train, size_lat, size_lon))
        scaled_labels_train[:, :, :, label_ix] = reshaped_scaled_train
        
    return scaled_features_train, scaled_features_predict, scaled_labels_train

# scale the training features and labels and prediction features
scaled_train_x, scaled_predict_x, scaled_train_y = scale_4d(train_x, predict_x, train_y, scalers_x, scalers_y)

### Define Keras models to use for prediciton
We'll define two neural network models using the Keras library with TensorFlow as its backend. One of these models will contain only simple densely connected layers, and another will contain a both convolutional layer and a densely connected layer. We'll use both of these for prediction of labels corresponding to the results of NCAR CAM model runs involving computation of the Held-Suarez test case. Initially we'll focus on the input feature variables PS, T, U, and V and the output label PTTEND.

##### Dense layer-only model

In [18]:
from keras.models import Sequential
from keras.layers import Dense

# define the model
dense_model = Sequential()

# add a fully-connected hidden layer with the same number of neurons as input attributes (features)
dense_model.add(Dense(len(features), input_dim=len(features), activation='relu'))

# add a fully-connected hidden layer with the twice the number of neurons as input attributes (features)
dense_model.add(Dense(len(features) * 2, activation='relu'))

# output layer uses no activation function since we are interested
# in predicting numerical values directly without transform
dense_model.add(Dense(len(labels)))

# compile the model using the ADAM optimization algorithm and a mean squared error loss function
dense_model.compile(optimizer='adam', loss='mse')

# display some summary information
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 69
Trainable params: 69
Non-trainable params: 0
_________________________________________________________________


##### Convolutional layer model

In [19]:
from keras.models import Sequential
from keras.layers import Conv3D, Dense

# define the model
cnn_model = Sequential()

# add an initial 3-D convolutional layer
cnn_model.add(Conv3D(filters=32,
                     kernel_size=(3, 3, 3),
                     activation="relu",
                     data_format="channels_last",
                     input_shape=(size_times_train, size_lat, size_lon, len(features)),
                     padding='same'))

# add a fully-connected hidden layer with twice the number of neurons as input attributes (features)
cnn_model.add(Dense(len(features) * 2, activation='relu'))

# output layer uses no activation function since we are interested
# in predicting numerical values directly without transform
cnn_model.add(Dense(len(labels)))

# compile the model using the ADAM optimization algorithm and a mean squared error loss function
cnn_model.compile(optimizer='adam', loss='mse')

# display some summary information
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 720, 12, 23, 32)   3488      
_________________________________________________________________
dense_9 (Dense)              (None, 720, 12, 23, 8)    264       
_________________________________________________________________
dense_10 (Dense)             (None, 720, 12, 23, 1)    9         
Total params: 3,761
Trainable params: 3,761
Non-trainable params: 0
_________________________________________________________________


##### Reshape data for convolutional model input

In [31]:
shape_x = (1, ) + scaled_train_x.shape
shape_y = (1, ) + scaled_train_y.shape
train_x_cnn = np.reshape(scaled_train_x, newshape=shape_x)
train_y_cnn = np.reshape(scaled_train_y, newshape=shape_y)
predict_x_cnn = np.reshape(scaled_predict_x, newshape=shape_x)

##### Train the convolutional model (for the first level)

In [32]:
cnn_model.fit(train_x_cnn, train_y_cnn, shuffle=True, epochs=8, verbose=2)

Epoch 1/8
 - 0s - loss: 0.0261
Epoch 2/8
 - 0s - loss: 0.0233
Epoch 3/8
 - 0s - loss: 0.0226
Epoch 4/8
 - 0s - loss: 0.0226
Epoch 5/8
 - 0s - loss: 0.0213
Epoch 6/8
 - 0s - loss: 0.0200
Epoch 7/8
 - 0s - loss: 0.0201
Epoch 8/8
 - 0s - loss: 0.0181


##### Reshape data for dense model input

In [33]:
shape_x = (size_times_train * size_lat * size_lon, len(features))
shape_y = (size_times_train * size_lat * size_lon, len(labels))
train_x_dense = np.reshape(scaled_train_x, newshape=shape_x)
train_y_dense = np.reshape(scaled_train_y, newshape=shape_y)
predict_x_dense = np.reshape(scaled_predict_x, newshape=shape_x)

##### Train the dense layers model (for the first level)

In [34]:
dense_model.fit(train_x_dense, train_y_dense, shuffle=True, epochs=8, verbose=2)

Epoch 1/8
 - 16s - loss: 9.3371e-08
Epoch 2/8
 - 16s - loss: 9.5670e-08
Epoch 3/8
 - 17s - loss: 1.0026e-07
Epoch 4/8
 - 17s - loss: 9.2125e-08
Epoch 5/8
 - 16s - loss: 9.3517e-08
Epoch 6/8
 - 17s - loss: 9.3825e-08
Epoch 7/8
 - 17s - loss: 9.3823e-08
Epoch 8/8
 - 17s - loss: 9.1776e-08


##### Allocate an array to contain predicted values

In [36]:
prediction_cnn = np.empty(shape=(size_times_predict, size_lev, size_lat, size_lon))
prediction_dense = np.empty(shape=(size_times_predict, size_lev, size_lat, size_lon))

In [41]:
predict_y_scaled_cnn = cnn_model.predict(predict_x_cnn, verbose=1)

1/1 [==============================] - 0s 64ms/step


In [42]:
predict_y_scaled_cnn.shape

(1, 720, 12, 23, 1)

### Prediction loop
We'll loop over each level, taking data for the full grid at that level (all time steps) and training the model with those feature and labels. We'll then use the fitted model to predict label values for the level using input feature values from a different time series. 

In [ ]:
# loop over all levels
for lev in range(size_lev):
    
    print("Training/predicting for level {level}".format(level=lev))
    
    # get the features and labels for training
    train_x, train_y = extract_features_labels(netcdf_features_train[0],
                                               netcdf_labels_train[0],
                                               features,
                                               labels,
                                               lev)
    
    # get the features for prediction
    predict_x = extract_data_array(xr.open_dataset(netcdf_features_predict[0]),
                                   features,
                                   lev)

    # scale the data between 0 and 1
    scalers_x = [MinMaxScaler(feature_range=(0, 1))] * len(features)
    scalers_y = [MinMaxScaler(feature_range=(0, 1))] * len(labels)
    scaled_train_x, scaled_predict_x, scaled_train_y = scale_4d(train_x, 
                                                                predict_x, 
                                                                train_y, 
                                                                scalers_x, 
                                                                scalers_y)
    
    # reshape the data for convolutional model input
    shape_x = (1, ) + scaled_train_x.shape
    shape_y = (1, ) + scaled_train_y.shape
    train_x_cnn = np.reshape(scaled_train_x, newshape=shape_x)
    train_y_cnn = np.reshape(scaled_train_y, newshape=shape_y)
    predict_x_cnn = np.reshape(scaled_predict_x, newshape=shape_x)
    
    # reshape the data for dense layer model input
    shape_x = (size_times_train * size_lat * size_lon, len(features))
    shape_y = (size_times_train * size_lat * size_lon, len(labels))
    train_x_dense = np.reshape(scaled_train_x, newshape=shape_x)
    train_y_dense = np.reshape(scaled_train_y, newshape=shape_y)
    predict_x_dense = np.reshape(scaled_predict_x, newshape=shape_x)
    
    # train the models
    cnn_model.fit(train_x_cnn, train_y_cnn, shuffle=True, epochs=8, verbose=2)
    dense_model.fit(train_x_dense, train_y_dense, shuffle=True, epochs=8, verbose=2)
    
    # use the fitted models to make predictions
    predict_y_scaled_cnn = cnn_model.predict(predict_x_cnn, verbose=1)
    predict_y_scaled_dense = dense_model.predict(predict_x_dense, verbose=1)

    # reverse the scaling of the predicted values
    # TODO below assumes a single label, will need modification for multiple labels
    scaler = scalers_y[0]  # assumes the label scaler was fitted in scale_4d() and side effect carried through
        
    # output from the dense model is 2-D, good for scaler input
    unscaled_predict_y_dense = scaler.inverse_transform(predict_y_scaled_dense)
        
    # output from CNN model is 5-D, so we'll flatten first to make it amenable to scaling
    unscaled_predict_y_cnn = scaler.inverse_transform(predict_y_scaled_cnn.flatten().reshape(-1, 1))
    
    # reshape data so it's compatible with assignment into prediction arrays
    level_shape = (size_times_predict, size_lat, size_lon)
    prediction_cnn[:, lev, :, :] = np.reshape(unscaled_predict_y_cnn, newshape=level_shape)
    prediction_dense[:, lev, :, :] = np.reshape(unscaled_predict_y_dense, newshape=level_shape)

Training/predicting for level 0
Epoch 1/8
 - 0s - loss: 0.0176
Epoch 2/8
 - 0s - loss: 0.0167
Epoch 3/8
 - 0s - loss: 0.0157
Epoch 4/8
 - 0s - loss: 0.0156
Epoch 5/8
 - 0s - loss: 0.0144
Epoch 6/8
 - 0s - loss: 0.0144
Epoch 7/8
 - 0s - loss: 0.0133
Epoch 8/8
 - 0s - loss: 0.0133
Epoch 1/8
 - 18s - loss: 9.8282e-08
Epoch 2/8
 - 17s - loss: 9.0407e-08
Epoch 3/8
 - 17s - loss: 9.3840e-08
Epoch 4/8
 - 17s - loss: 9.9851e-08
Epoch 5/8
 - 18s - loss: 9.0942e-08
Epoch 6/8
 - 16s - loss: 9.1971e-08
Epoch 7/8
 - 16s - loss: 9.4488e-08
Epoch 8/8
 - 17s - loss: 9.0844e-08
198720/198720 [==============================] - 19s 96us/step
Training/predicting for level 1
Epoch 1/8
 - 0s - loss: 0.0212
Epoch 2/8
 - 0s - loss: 0.0119
Epoch 3/8
 - 0s - loss: 0.0194
Epoch 4/8
 - 0s - loss: 0.0192
Epoch 5/8
 - 0s - loss: 0.0122
Epoch 6/8
 - 0s - loss: 0.0116
Epoch 7/8
 - 0s - loss: 0.0163
Epoch 8/8
 - 0s - loss: 0.0118
Epoch 1/8
 - 17s - loss: 1.2190e-07
Epoch 2/8
 - 18s - loss: 1.0547e-07
Epoch 3/8
 - 17s 

198720/198720 [==============================] - 20s 99us/step
Training/predicting for level 12
Epoch 1/8
 - 0s - loss: 0.0035
Epoch 2/8
 - 0s - loss: 0.0031
Epoch 3/8
 - 0s - loss: 0.0026
Epoch 4/8
 - 0s - loss: 0.0022
Epoch 5/8
 - 0s - loss: 0.0019
Epoch 6/8
 - 0s - loss: 0.0016
Epoch 7/8
 - 0s - loss: 0.0013
Epoch 8/8
 - 0s - loss: 0.0010
Epoch 1/8
 - 17s - loss: 2.2537e-04
Epoch 2/8
 - 17s - loss: 2.2365e-04
Epoch 3/8
 - 17s - loss: 2.2284e-04
Epoch 4/8
 - 18s - loss: 2.2234e-04
Epoch 5/8
 - 18s - loss: 2.2176e-04
Epoch 6/8
 - 18s - loss: 2.2144e-04
Epoch 7/8
 - 17s - loss: 2.2081e-04
Epoch 8/8
 - 18s - loss: 2.2067e-04
198720/198720 [==============================] - 19s 97us/step
Training/predicting for level 13
Epoch 1/8
 - 0s - loss: 0.0010
Epoch 2/8
 - 0s - loss: 8.5609e-04
Epoch 3/8
 - 0s - loss: 7.0706e-04
Epoch 4/8
 - 0s - loss: 5.8965e-04
Epoch 5/8
 - 0s - loss: 5.0060e-04
Epoch 6/8
 - 0s - loss: 4.3634e-04
Epoch 7/8
 - 0s - loss: 3.9324e-04
Epoch 8/8
 - 0s - loss: 3.6768e

### Output of predicted values as NetCDF
At this point the entire dataset has been predicted and the predicted values are in arrays that we can add to an xarray DataSet that we'll then write as NetCDF. We will copy the prediction features dataset sicne this has all the coordinate variables and attributes we'll want in the predicted labels dataset, then we'll add the predicted values arrays as variables to the dataset before writing it as NetCDF.

In [ ]:
# copy the prediction features dataset since the predicted label(s) should share the same coordinates, etc.
ds_predict_labels = ds_predict_features.copy(deep=True)

# remove all non-label data variables from the predictions dataset
for var in ds_predict_labels.data_vars:
    if var not in labels:
        ds_predict_labels = ds_predict_labels.drop(var)

# create new variables to contain the predicted labels, assign these into the prediction dataset
predicted_label_var = xr.Variable(dims=('time', 'lev', 'lat', 'lon'),
                                  data=prediction_cnn,
                                  attrs=ds_learn_labels[labels[0]].attrs)
ds_predict_labels[labels[0] + "_cnn"] = predicted_label_var
predicted_label_var = xr.Variable(dims=('time', 'lev', 'lat', 'lon'),
                                  data=prediction_dense,
                                  attrs=ds_learn_labels[labels[0]].attrs)
ds_predict_labels[labels[0] + "_dense"] = predicted_label_var

# write the predicted label(s)' dataset as a NetCDF file
ds_predict_labels.to_netcdf(netcdf_predict[0])